### this notebook (analysis.ipynb) contains R code that analyzes the data from the simulation 
### it answers the following questions
### 1. what are the effect of different ad campaigns on purchase probability and sales?
### 2. how does ad type affects funnel progression?
### 3. how does ad type affects purchase probability and sales for users in the purchase stage?
### 4. what are the ROIs of different ad campaigns 
### 5. how effective is the full-funnel campaign when funnel stage is predicted with high, medium, and low accuracy?
### for more advanced treatment, students can combine the simulation and analysis notebooks to explore what simulation parameters affect campaign performance, for instance, how results change for new vs well-known brands (i.e., when most users start from not aware vs aware stage)

In [1]:
suppressMessages(library(tidyverse))

In [2]:
df = read.csv('data.csv')
df = arrange(df, campaign_type, user_id, visit)

In [3]:
# control group as the baseline
df$campaign_type = as.factor(df$campaign_type)
df$campaign_type <- relevel(df$campaign_type, ref = 'control')

In [4]:
# no ad as the baseline
df$ad_type = as.factor(df$ad_type)
df$ad_type <- relevel(df$ad_type, ref = 'none')

In [5]:
# inspect the data structure 
head(df)

,user_id,current_funnel_stage,next_funnel_stage,ad_type,purchase,sales,date,campaign_type,visit
,<int>,<chr>,<chr>,<fct>,<int>,<int>,<chr>,<fct>,<int>
1,30001,purchase,purchase,branding,0,0,2025-05-14,brand_plus_performance,1
2,30001,purchase,purchase,branding,0,0,2025-05-18,brand_plus_performance,2
3,30001,purchase,purchase,performance,0,0,2025-05-25,brand_plus_performance,3
4,30001,purchase,purchase,performance,1,100,2025-06-01,brand_plus_performance,4
5,30002,not aware,not aware,branding,0,0,2025-04-29,brand_plus_performance,1
6,30002,not aware,aware,branding,0,0,2025-05-06,brand_plus_performance,2


In [6]:
# all possible funnel stages
unique(as.character(df$current_funnel_stage))

[1] "purchase"  "not aware" "aware"     "consider"

In [7]:
# all ad types
unique(as.character(df$ad_type))

[1] "branding"    "performance" "none"

In [8]:
# all campaign types
unique(as.character(df$campaign_type))

[1] "brand_plus_performance" "branding"               "control"               
[4] "full_funnel"            "performance"

#### 1. overall effect of campaign on purchase probability and sales

In [9]:
# purchase probability as the outcome (all campaigns)
# intercept shows purchase probability without any ad
# data is aggregated from user-visit level to user level
model = lm(purchase ~ campaign_type, 
           df %>% 
           group_by(user_id) %>% 
           mutate(purchase = sum(purchase)) %>% 
           distinct(user_id, .keep_all = T))
summary(model)


Call:
lm(formula = purchase ~ campaign_type, data = df %>% group_by(user_id) %>% 
    mutate(purchase = sum(purchase)) %>% distinct(user_id, .keep_all = T))

Residuals:
    Min      1Q  Median      3Q     Max 
-0.0953 -0.0834 -0.0638 -0.0423  0.9577 

Coefficients:
                                     Estimate Std. Error t value Pr(>|t|)    
(Intercept)                          0.043000   0.002466  17.436  < 2e-16 ***
campaign_typebrand_plus_performance  0.020800   0.003488   5.964 2.48e-09 ***
campaign_typebranding               -0.000700   0.003488  -0.201    0.841    
campaign_typefull_funnel             0.052300   0.003488  14.996  < 2e-16 ***
campaign_typeperformance             0.040400   0.003488  11.584  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.2466 on 49995 degrees of freedom
Multiple R-squared:  0.007364,	Adjusted R-squared:  0.007285 
F-statistic: 92.73 on 4 and 49995 DF,  p-value: < 2.2e-16


In [10]:
# purchase probability as the outcome (directly comparing any two campaigns)
model = lm(purchase ~ campaign_type, 
           df %>% 
           group_by(user_id) %>% 
           mutate(purchase = sum(purchase)) %>% 
           distinct(user_id, .keep_all = T) %>%
           filter(campaign_type %in% c('brand_plus_performance', 'full_funnel'))) 
summary(model)


Call:
lm(formula = purchase ~ campaign_type, data = df %>% group_by(user_id) %>% 
    mutate(purchase = sum(purchase)) %>% distinct(user_id, .keep_all = T) %>% 
    filter(campaign_type %in% c("brand_plus_performance", "full_funnel")))

Residuals:
    Min      1Q  Median      3Q     Max 
-0.0953 -0.0953 -0.0638 -0.0638  0.9362 

Coefficients:
                         Estimate Std. Error t value Pr(>|t|)    
(Intercept)              0.063800   0.002702  23.617   <2e-16 ***
campaign_typefull_funnel 0.031500   0.003820   8.245   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.2702 on 19998 degrees of freedom
Multiple R-squared:  0.003388,	Adjusted R-squared:  0.003338 
F-statistic: 67.98 on 1 and 19998 DF,  p-value: < 2.2e-16


In [11]:
# purchase probability as the outcome (directly comparing any two campaigns)
model = lm(purchase ~ campaign_type, 
           df %>% 
           group_by(user_id) %>% 
           mutate(purchase = sum(purchase)) %>% 
           distinct(user_id, .keep_all = T) %>%
           filter(campaign_type %in% c('performance', 'full_funnel')))
summary(model)


Call:
lm(formula = purchase ~ campaign_type, data = df %>% group_by(user_id) %>% 
    mutate(purchase = sum(purchase)) %>% distinct(user_id, .keep_all = T) %>% 
    filter(campaign_type %in% c("performance", "full_funnel")))

Residuals:
    Min      1Q  Median      3Q     Max 
-0.0953 -0.0953 -0.0834 -0.0834  0.9166 

Coefficients:
                          Estimate Std. Error t value Pr(>|t|)    
(Intercept)               0.095300   0.002852   33.41  < 2e-16 ***
campaign_typeperformance -0.011900   0.004033   -2.95  0.00318 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.2852 on 19998 degrees of freedom
Multiple R-squared:  0.0004351,	Adjusted R-squared:  0.0003851 
F-statistic: 8.705 on 1 and 19998 DF,  p-value: 0.003177


In [12]:
# sales as the outcome (all campaigns)
# intercept shows average sales without any ad
# data is aggregated from user-visit level to user level
model = lm(sales ~ campaign_type, 
           df %>% 
           group_by(user_id) %>% 
           mutate(sales = sum(sales)) %>% 
           distinct(user_id, .keep_all = T))
summary(model)


Call:
lm(formula = sales ~ campaign_type, data = df %>% group_by(user_id) %>% 
    mutate(sales = sum(sales)) %>% distinct(user_id, .keep_all = T))

Residuals:
   Min     1Q Median     3Q    Max 
 -9.53  -8.34  -6.38  -4.23  95.77 

Coefficients:
                                    Estimate Std. Error t value Pr(>|t|)    
(Intercept)                           4.3000     0.2466  17.436  < 2e-16 ***
campaign_typebrand_plus_performance   2.0800     0.3488   5.964 2.48e-09 ***
campaign_typebranding                -0.0700     0.3488  -0.201    0.841    
campaign_typefull_funnel              5.2300     0.3488  14.996  < 2e-16 ***
campaign_typeperformance              4.0400     0.3488  11.584  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 24.66 on 49995 degrees of freedom
Multiple R-squared:  0.007364,	Adjusted R-squared:  0.007285 
F-statistic: 92.73 on 4 and 49995 DF,  p-value: < 2.2e-16


In [13]:
# sales as the outcome (directly comparing any two campaigns)
model = lm(sales ~ campaign_type, 
           df %>% 
           group_by(user_id) %>% 
           mutate(sales = sum(sales)) %>% 
           distinct(user_id, .keep_all = T) %>%
           filter(campaign_type %in% c('brand_plus_performance', 'full_funnel')))
summary(model)


Call:
lm(formula = sales ~ campaign_type, data = df %>% group_by(user_id) %>% 
    mutate(sales = sum(sales)) %>% distinct(user_id, .keep_all = T) %>% 
    filter(campaign_type %in% c("brand_plus_performance", "full_funnel")))

Residuals:
   Min     1Q Median     3Q    Max 
 -9.53  -9.53  -6.38  -6.38  93.62 

Coefficients:
                         Estimate Std. Error t value Pr(>|t|)    
(Intercept)                6.3800     0.2702  23.617   <2e-16 ***
campaign_typefull_funnel   3.1500     0.3820   8.245   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 27.02 on 19998 degrees of freedom
Multiple R-squared:  0.003388,	Adjusted R-squared:  0.003338 
F-statistic: 67.98 on 1 and 19998 DF,  p-value: < 2.2e-16


In [14]:
# sales as the outcome (directly comparing any two campaigns)
model = lm(sales ~ campaign_type, 
           df %>% 
           group_by(user_id) %>% 
           mutate(sales = sum(sales)) %>% 
           distinct(user_id, .keep_all = T) %>%
           filter(campaign_type %in% c('performance', 'full_funnel')))
summary(model)


Call:
lm(formula = sales ~ campaign_type, data = df %>% group_by(user_id) %>% 
    mutate(sales = sum(sales)) %>% distinct(user_id, .keep_all = T) %>% 
    filter(campaign_type %in% c("performance", "full_funnel")))

Residuals:
   Min     1Q Median     3Q    Max 
 -9.53  -9.53  -8.34  -8.34  91.66 

Coefficients:
                         Estimate Std. Error t value Pr(>|t|)    
(Intercept)                9.5300     0.2852   33.41  < 2e-16 ***
campaign_typeperformance  -1.1900     0.4033   -2.95  0.00318 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 28.52 on 19998 degrees of freedom
Multiple R-squared:  0.0004351,	Adjusted R-squared:  0.0003851 
F-statistic: 8.705 on 1 and 19998 DF,  p-value: 0.003177


#### 2. effect of ad type on funnel progression

In [15]:
df$stage_progress = ifelse(df$current_funnel_stage == df$next_funnel_stage, 0, 1)

In [16]:
model = lm(stage_progress ~ ad_type, 
          filter(df, current_funnel_stage == 'not aware'))
summary(model)


Call:
lm(formula = stage_progress ~ ad_type, data = filter(df, current_funnel_stage == 
    "not aware"))

Residuals:
     Min       1Q   Median       3Q      Max 
-0.39943 -0.39943 -0.09907 -0.09669  0.90331 

Coefficients:
                   Estimate Std. Error t value Pr(>|t|)    
(Intercept)        0.096691   0.002519  38.391   <2e-16 ***
ad_typebranding    0.302741   0.003168  95.570   <2e-16 ***
ad_typeperformance 0.002376   0.003405   0.698    0.485    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.3933 on 95756 degrees of freedom
Multiple R-squared:  0.1263,	Adjusted R-squared:  0.1263 
F-statistic:  6922 on 2 and 95756 DF,  p-value: < 2.2e-16


In [17]:
model = lm(stage_progress ~ ad_type, 
          filter(df, current_funnel_stage == 'aware'))
summary(model)


Call:
lm(formula = stage_progress ~ ad_type, data = filter(df, current_funnel_stage == 
    "aware"))

Residuals:
     Min       1Q   Median       3Q      Max 
-0.29872 -0.29872 -0.09811 -0.09811  0.90189 

Coefficients:
                    Estimate Std. Error t value Pr(>|t|)    
(Intercept)         0.104838   0.004696  22.325   <2e-16 ***
ad_typebranding    -0.006726   0.005439  -1.237    0.216    
ad_typeperformance  0.193884   0.005255  36.897   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.3873 on 53679 degrees of freedom
Multiple R-squared:  0.06189,	Adjusted R-squared:  0.06186 
F-statistic:  1771 on 2 and 53679 DF,  p-value: < 2.2e-16


In [18]:
model = lm(stage_progress ~ ad_type, 
          filter(df, current_funnel_stage == 'consider'))
summary(model)


Call:
lm(formula = stage_progress ~ ad_type, data = filter(df, current_funnel_stage == 
    "consider"))

Residuals:
     Min       1Q   Median       3Q      Max 
-0.30193 -0.30193 -0.10162 -0.09758  0.90242 

Coefficients:
                    Estimate Std. Error t value Pr(>|t|)    
(Intercept)         0.101624   0.006234  16.302   <2e-16 ***
ad_typebranding    -0.004049   0.007869  -0.514    0.607    
ad_typeperformance  0.200303   0.007072  28.325   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.3974 on 25076 degrees of freedom
Multiple R-squared:  0.0602,	Adjusted R-squared:  0.06012 
F-statistic: 803.1 on 2 and 25076 DF,  p-value: < 2.2e-16


#### 3. effect of ad type on purchase probability and sales

In [19]:
# purchase probability as the outcome 
model = lm(purchase ~ ad_type, 
          filter(df, current_funnel_stage == 'purchase'))
summary(model)


Call:
lm(formula = purchase ~ ad_type, data = filter(df, current_funnel_stage == 
    "purchase"))

Residuals:
    Min      1Q  Median      3Q     Max 
-0.2031 -0.2031 -0.1061 -0.1035  0.8965 

Coefficients:
                    Estimate Std. Error t value Pr(>|t|)    
(Intercept)         0.106068   0.005649  18.777   <2e-16 ***
ad_typebranding    -0.002585   0.007327  -0.353    0.724    
ad_typeperformance  0.097024   0.006609  14.681   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.3597 on 20989 degrees of freedom
Multiple R-squared:  0.01839,	Adjusted R-squared:  0.0183 
F-statistic: 196.7 on 2 and 20989 DF,  p-value: < 2.2e-16


In [20]:
# sales as the outcome 
model = lm(sales ~ ad_type, 
          filter(df, current_funnel_stage == 'purchase'))
summary(model)


Call:
lm(formula = sales ~ ad_type, data = filter(df, current_funnel_stage == 
    "purchase"))

Residuals:
   Min     1Q Median     3Q    Max 
-20.31 -20.31 -10.61 -10.35  89.65 

Coefficients:
                   Estimate Std. Error t value Pr(>|t|)    
(Intercept)         10.6068     0.5649  18.777   <2e-16 ***
ad_typebranding     -0.2585     0.7327  -0.353    0.724    
ad_typeperformance   9.7024     0.6609  14.681   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 35.97 on 20989 degrees of freedom
Multiple R-squared:  0.01839,	Adjusted R-squared:  0.0183 
F-statistic: 196.7 on 2 and 20989 DF,  p-value: < 2.2e-16


#### 4. ROI

In [21]:
# cpm (cost per thousand impressions) for branding ad, can be changed 
cpm = 30 
# cpa (cost per action/purchase) for performance ad, can be changed
cpa = 10

In [22]:
# cost of branding campaign
branding_ad_cost = cpm/1000 * df %>% filter(campaign_type == 'branding', ad_type == 'branding') %>% nrow
performance_ad_cost = 0
cost_branding = branding_ad_cost + performance_ad_cost

print(paste("branding ad cost:", branding_ad_cost))
print(paste("performance ad cost:", performance_ad_cost))
print(paste("total cost:", cost_branding))

[1] "branding ad cost: 1180.77"
[1] "performance ad cost: 0"
[1] "total cost: 1180.77"


In [23]:
# cost of performance campaign
branding_ad_cost = 0
performance_ad_cost = cpa * df %>% filter(campaign_type == 'performance', ad_type == 'performance') %>% pull(purchase) %>% sum
cost_performance = branding_ad_cost + performance_ad_cost

print(paste("branding ad cost:", branding_ad_cost))
print(paste("performance ad cost:", performance_ad_cost))
print(paste("total cost:", cost_performance))

[1] "branding ad cost: 0"
[1] "performance ad cost: 8340"
[1] "total cost: 8340"


In [24]:
# cost of brand-plus-performance campaign
branding_ad_cost = cpm/1000 * df %>% filter(campaign_type == 'brand_plus_performance', ad_type == 'branding') %>% nrow
performance_ad_cost = cpa * df %>% filter(campaign_type == 'brand_plus_performance', ad_type == 'performance') %>% pull(purchase) %>% sum
cost_brand_plus_performance = branding_ad_cost + performance_ad_cost

print(paste("branding ad cost:", branding_ad_cost))
print(paste("performance ad cost:", performance_ad_cost))
print(paste("total cost:", cost_brand_plus_performance))

[1] "branding ad cost: 597.15"
[1] "performance ad cost: 4460"
[1] "total cost: 5057.15"


In [25]:
# cost of full-funnel campaign
branding_ad_cost = cpm/1000 * df %>% filter(campaign_type == 'full_funnel', ad_type == 'branding') %>% nrow
performance_ad_cost = cpa * df %>% filter(campaign_type == 'full_funnel', ad_type == 'performance') %>% pull(purchase) %>% sum
cost_full_funnel = branding_ad_cost + performance_ad_cost

print(paste("branding ad cost:", branding_ad_cost))
print(paste("performance ad cost:", performance_ad_cost))
print(paste("total cost:", cost_full_funnel))

[1] "branding ad cost: 460.14"
[1] "performance ad cost: 9530"
[1] "total cost: 9990.14"


In [26]:
# ROI of branding campaign
roi_branding = ((df %>% filter(campaign_type == 'branding') %>% pull(sales) %>% sum - 
                 df %>% filter(campaign_type == 'control') %>% pull(sales) %>% sum) - cost_branding) / cost_branding
print(paste("branding campaign ROI:", roi_branding))

[1] "branding campaign ROI: -1.59283349001076"


In [27]:
# ROI of performance campaign
roi_performance = ((df %>% filter(campaign_type == 'performance') %>% pull(sales) %>% sum - 
                    df %>% filter(campaign_type == 'control') %>% pull(sales) %>% sum) - cost_performance) / cost_performance
print(paste("performance campaign ROI:", roi_performance))

[1] "performance campaign ROI: 3.84412470023981"


In [28]:
# ROI of brand-plus-performance campaign
roi_brand_plus_performance = ((df %>% filter(campaign_type == 'brand_plus_performance') %>% pull(sales) %>% sum - 
                    df %>% filter(campaign_type == 'control') %>% pull(sales) %>% sum) - cost_brand_plus_performance) / cost_brand_plus_performance
print(paste("brand-plus-performance campaign ROI:", roi_brand_plus_performance))

[1] "brand-plus-performance campaign ROI: 3.11298854097664"


In [29]:
# ROI of full-funnel campaign
roi_full_funnel = ((df %>% filter(campaign_type == 'full_funnel') %>% pull(sales) %>% sum - 
                    df %>% filter(campaign_type == 'control') %>% pull(sales) %>% sum) - cost_full_funnel) / cost_full_funnel
print(paste("full-funnel campaign ROI:", roi_full_funnel))

[1] "full-funnel campaign ROI: 4.23516186960343"


#### 5. adding a full_funnel group where funnel stages are predicted instead of known

##### high prediction accuracy (90%)

In [30]:
df_predicted = read.csv('data_predicted_high.csv')

In [31]:
df_predicted$stage_progress = ifelse(df_predicted$current_funnel_stage == df_predicted$next_funnel_stage, 0, 1)

In [32]:
df_predicted = rbind(df_predicted, 
                     df %>% mutate(current_funnel_stage_predicted = current_funnel_stage, # add two predicted funnel stage columns in other conditions so we can stack them up 
                                  next_funnel_stage_predicted = next_funnel_stage))
df_predicted = arrange(df_predicted, campaign_type, user_id, visit)

In [33]:
# control group as the baseline
df_predicted$campaign_type = as.factor(df_predicted$campaign_type)
df_predicted$campaign_type <- relevel(df_predicted$campaign_type, ref = 'control')

In [34]:
# no ad as the baseline
df_predicted$ad_type = as.factor(df_predicted$ad_type)
df_predicted$ad_type <- relevel(df_predicted$ad_type, ref = 'none')

In [35]:
head(df_predicted)

,user_id,current_funnel_stage,current_funnel_stage_predicted,next_funnel_stage,next_funnel_stage_predicted,ad_type,purchase,sales,date,campaign_type,visit,stage_progress
,<int>,<chr>,<chr>,<chr>,<chr>,<fct>,<int>,<int>,<chr>,<fct>,<int>,<dbl>
1,30001,purchase,purchase,purchase,purchase,branding,0,0,2025-05-14,brand_plus_performance,1,0
2,30001,purchase,purchase,purchase,purchase,branding,0,0,2025-05-18,brand_plus_performance,2,0
3,30001,purchase,purchase,purchase,purchase,performance,0,0,2025-05-25,brand_plus_performance,3,0
4,30001,purchase,purchase,purchase,purchase,performance,1,100,2025-06-01,brand_plus_performance,4,0
5,30002,not aware,not aware,not aware,not aware,branding,0,0,2025-04-29,brand_plus_performance,1,0
6,30002,not aware,not aware,aware,aware,branding,0,0,2025-05-06,brand_plus_performance,2,1


In [36]:
# purchase probability as the outcome (all campaigns)
# intercept shows average probability without any ad
# data is aggregated from user-visit level to user level
model = lm(purchase ~ campaign_type, 
           df_predicted %>% 
           group_by(user_id) %>% 
           mutate(purchase = sum(purchase)) %>% 
           distinct(user_id, .keep_all = T))
summary(model)


Call:
lm(formula = purchase ~ campaign_type, data = df_predicted %>% 
    group_by(user_id) %>% mutate(purchase = sum(purchase)) %>% 
    distinct(user_id, .keep_all = T))

Residuals:
    Min      1Q  Median      3Q     Max 
-0.0953 -0.0909 -0.0638 -0.0430  0.9577 

Coefficients:
                                     Estimate Std. Error t value Pr(>|t|)    
(Intercept)                          0.043000   0.002539  16.937  < 2e-16 ***
campaign_typebrand_plus_performance  0.020800   0.003590   5.793 6.94e-09 ***
campaign_typebranding               -0.000700   0.003590  -0.195    0.845    
campaign_typefull_funnel             0.052300   0.003590  14.567  < 2e-16 ***
campaign_typefull_funnel_predicted   0.047900   0.003590  13.341  < 2e-16 ***
campaign_typeperformance             0.040400   0.003590  11.252  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.2539 on 59994 degrees of freedom
Multiple R-squared:  0.007166,	Adjusted R-sq

In [37]:
# purchase probability as the outcome (directly comparing two campaigns)

model = lm(purchase ~ campaign_type, 
           df_predicted %>% 
           group_by(user_id) %>% 
           mutate(purchase = sum(purchase)) %>% 
           distinct(user_id, .keep_all = T) %>%
           filter(campaign_type %in% c('full_funnel_predicted', 'full_funnel')))
summary(model)


Call:
lm(formula = purchase ~ campaign_type, data = df_predicted %>% 
    group_by(user_id) %>% mutate(purchase = sum(purchase)) %>% 
    distinct(user_id, .keep_all = T) %>% filter(campaign_type %in% 
    c("full_funnel_predicted", "full_funnel")))

Residuals:
    Min      1Q  Median      3Q     Max 
-0.0953 -0.0953 -0.0909 -0.0909  0.9091 

Coefficients:
                                    Estimate Std. Error t value Pr(>|t|)    
(Intercept)                         0.095300   0.002906  32.797   <2e-16 ***
campaign_typefull_funnel_predicted -0.004400   0.004109  -1.071    0.284    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.2906 on 19998 degrees of freedom
Multiple R-squared:  5.732e-05,	Adjusted R-squared:  7.322e-06 
F-statistic: 1.146 on 1 and 19998 DF,  p-value: 0.2843


In [38]:
# sales as the outcome (all campaigns)
# intercept shows average sales without any ad
# data is aggregated from user-visit level to user level
model = lm(sales ~ campaign_type, 
           df_predicted %>% 
           group_by(user_id) %>% 
           mutate(sales = sum(sales)) %>% 
           distinct(user_id, .keep_all = T))
summary(model)


Call:
lm(formula = sales ~ campaign_type, data = df_predicted %>% group_by(user_id) %>% 
    mutate(sales = sum(sales)) %>% distinct(user_id, .keep_all = T))

Residuals:
   Min     1Q Median     3Q    Max 
 -9.53  -9.09  -6.38  -4.30  95.77 

Coefficients:
                                    Estimate Std. Error t value Pr(>|t|)    
(Intercept)                           4.3000     0.2539  16.937  < 2e-16 ***
campaign_typebrand_plus_performance   2.0800     0.3590   5.793 6.94e-09 ***
campaign_typebranding                -0.0700     0.3590  -0.195    0.845    
campaign_typefull_funnel              5.2300     0.3590  14.567  < 2e-16 ***
campaign_typefull_funnel_predicted    4.7900     0.3590  13.341  < 2e-16 ***
campaign_typeperformance              4.0400     0.3590  11.252  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 25.39 on 59994 degrees of freedom
Multiple R-squared:  0.007166,	Adjusted R-squared:  0.007083 
F-statistic:  

In [39]:
# sales as the outcome (directly comparing two campaigns)

model = lm(sales ~ campaign_type, 
           df_predicted %>% 
           group_by(user_id) %>% 
           mutate(sales = sum(sales)) %>% 
           distinct(user_id, .keep_all = T) %>%
           filter(campaign_type %in% c('full_funnel_predicted', 'full_funnel')))
summary(model)


Call:
lm(formula = sales ~ campaign_type, data = df_predicted %>% group_by(user_id) %>% 
    mutate(sales = sum(sales)) %>% distinct(user_id, .keep_all = T) %>% 
    filter(campaign_type %in% c("full_funnel_predicted", "full_funnel")))

Residuals:
   Min     1Q Median     3Q    Max 
 -9.53  -9.53  -9.09  -9.09  90.91 

Coefficients:
                                   Estimate Std. Error t value Pr(>|t|)    
(Intercept)                          9.5300     0.2906  32.797   <2e-16 ***
campaign_typefull_funnel_predicted  -0.4400     0.4109  -1.071    0.284    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 29.06 on 19998 degrees of freedom
Multiple R-squared:  5.732e-05,	Adjusted R-squared:  7.322e-06 
F-statistic: 1.146 on 1 and 19998 DF,  p-value: 0.2843


In [40]:
# cost of predicted full-funnel campaign
branding_ad_cost = cpm/1000 * df_predicted %>% filter(campaign_type == 'full_funnel_predicted', ad_type == 'branding') %>% nrow
performance_ad_cost = cpa * df_predicted %>% filter(campaign_type == 'full_funnel_predicted', ad_type == 'performance') %>% pull(purchase) %>% sum
cost_full_funnel_predicted = branding_ad_cost + performance_ad_cost

print(paste("branding ad cost:", branding_ad_cost))
print(paste("performance ad cost:", performance_ad_cost))
print(paste("total cost:", cost_full_funnel_predicted))

[1] "branding ad cost: 458.73"
[1] "performance ad cost: 8950"
[1] "total cost: 9408.73"


In [41]:
# ROI of predicted full-funnel campaign
roi_full_funnel_predicted = ((df_predicted %>% filter(campaign_type == 'full_funnel_predicted') %>% pull(sales) %>% sum - 
                    df %>% filter(campaign_type == 'control') %>% pull(sales) %>% sum) - cost_full_funnel_predicted) / cost_full_funnel_predicted
print(paste("predicted full-funnel campaign ROI:", roi_full_funnel_predicted))

[1] "predicted full-funnel campaign ROI: 4.09101653464389"


##### medium prediction accuracy (60%)

In [42]:
df_predicted = read.csv('data_predicted_medium.csv')

In [43]:
df_predicted$stage_progress = ifelse(df_predicted$current_funnel_stage == df_predicted$next_funnel_stage, 0, 1)

In [44]:
df_predicted = rbind(df_predicted, 
                     df %>% mutate(current_funnel_stage_predicted = current_funnel_stage,
                                  next_funnel_stage_predicted = next_funnel_stage))
df_predicted = arrange(df_predicted, campaign_type, user_id, visit)

In [45]:
# control group as the baseline
df_predicted$campaign_type = as.factor(df_predicted$campaign_type)
df_predicted$campaign_type <- relevel(df_predicted$campaign_type, ref = 'control')

In [46]:
# no ad as the baseline
df_predicted$ad_type = as.factor(df_predicted$ad_type)
df_predicted$ad_type <- relevel(df_predicted$ad_type, ref = 'none')

In [47]:
# purchase probability as the outcome (all campaigns)
# intercept shows average probability without any ad
# data is aggregated from user-visit level to user level
model = lm(purchase ~ campaign_type, 
           df_predicted %>% 
           group_by(user_id) %>% 
           mutate(purchase = sum(purchase)) %>% 
           distinct(user_id, .keep_all = T))
summary(model)


Call:
lm(formula = purchase ~ campaign_type, data = df_predicted %>% 
    group_by(user_id) %>% mutate(purchase = sum(purchase)) %>% 
    distinct(user_id, .keep_all = T))

Residuals:
    Min      1Q  Median      3Q     Max 
-0.0953 -0.0839 -0.0638 -0.0430  0.9577 

Coefficients:
                                     Estimate Std. Error t value Pr(>|t|)    
(Intercept)                          0.043000   0.002520  17.065  < 2e-16 ***
campaign_typebrand_plus_performance  0.020800   0.003563   5.837 5.34e-09 ***
campaign_typebranding               -0.000700   0.003563  -0.196    0.844    
campaign_typefull_funnel             0.052300   0.003563  14.677  < 2e-16 ***
campaign_typefull_funnel_predicted   0.040900   0.003563  11.478  < 2e-16 ***
campaign_typeperformance             0.040400   0.003563  11.337  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.252 on 59994 degrees of freedom
Multiple R-squared:  0.006614,	Adjusted R-squ

In [48]:
# purchase probability as the outcome (directly comparing two campaigns)
model = lm(purchase ~ campaign_type, 
           df_predicted %>% 
           group_by(user_id) %>% 
           mutate(purchase = sum(purchase)) %>% 
           distinct(user_id, .keep_all = T) %>%
           filter(campaign_type %in% c('full_funnel_predicted', 'full_funnel')))
summary(model)


Call:
lm(formula = purchase ~ campaign_type, data = df_predicted %>% 
    group_by(user_id) %>% mutate(purchase = sum(purchase)) %>% 
    distinct(user_id, .keep_all = T) %>% filter(campaign_type %in% 
    c("full_funnel_predicted", "full_funnel")))

Residuals:
    Min      1Q  Median      3Q     Max 
-0.0953 -0.0953 -0.0839 -0.0839  0.9161 

Coefficients:
                                    Estimate Std. Error t value Pr(>|t|)    
(Intercept)                         0.095300   0.002856  33.372  < 2e-16 ***
campaign_typefull_funnel_predicted -0.011400   0.004039  -2.823  0.00476 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.2856 on 19998 degrees of freedom
Multiple R-squared:  0.0003983,	Adjusted R-squared:  0.0003483 
F-statistic: 7.968 on 1 and 19998 DF,  p-value: 0.004765


In [49]:
# sales as the outcome (all campaigns)
# intercept shows average sales without any ad
# data is aggregated from user-visit level to user level
model = lm(sales ~ campaign_type, 
           df_predicted %>% 
           group_by(user_id) %>% 
           mutate(sales = sum(sales)) %>% 
           distinct(user_id, .keep_all = T))
summary(model)


Call:
lm(formula = sales ~ campaign_type, data = df_predicted %>% group_by(user_id) %>% 
    mutate(sales = sum(sales)) %>% distinct(user_id, .keep_all = T))

Residuals:
   Min     1Q Median     3Q    Max 
 -9.53  -8.39  -6.38  -4.30  95.77 

Coefficients:
                                    Estimate Std. Error t value Pr(>|t|)    
(Intercept)                           4.3000     0.2520  17.065  < 2e-16 ***
campaign_typebrand_plus_performance   2.0800     0.3564   5.837 5.34e-09 ***
campaign_typebranding                -0.0700     0.3564  -0.196    0.844    
campaign_typefull_funnel              5.2300     0.3564  14.677  < 2e-16 ***
campaign_typefull_funnel_predicted    4.0900     0.3564  11.478  < 2e-16 ***
campaign_typeperformance              4.0400     0.3564  11.337  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 25.2 on 59994 degrees of freedom
Multiple R-squared:  0.006614,	Adjusted R-squared:  0.006531 
F-statistic: 79

In [50]:
# sales as the outcome (directly comparing two campaigns)
model = lm(sales ~ campaign_type, 
           df_predicted %>% 
           group_by(user_id) %>% 
           mutate(sales = sum(sales)) %>% 
           distinct(user_id, .keep_all = T) %>%
           filter(campaign_type %in% c('full_funnel_predicted', 'full_funnel')))
summary(model)


Call:
lm(formula = sales ~ campaign_type, data = df_predicted %>% group_by(user_id) %>% 
    mutate(sales = sum(sales)) %>% distinct(user_id, .keep_all = T) %>% 
    filter(campaign_type %in% c("full_funnel_predicted", "full_funnel")))

Residuals:
   Min     1Q Median     3Q    Max 
 -9.53  -9.53  -8.39  -8.39  91.61 

Coefficients:
                                   Estimate Std. Error t value Pr(>|t|)    
(Intercept)                          9.5300     0.2856  33.372  < 2e-16 ***
campaign_typefull_funnel_predicted  -1.1400     0.4039  -2.823  0.00476 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 28.56 on 19998 degrees of freedom
Multiple R-squared:  0.0003983,	Adjusted R-squared:  0.0003483 
F-statistic: 7.968 on 1 and 19998 DF,  p-value: 0.004765


In [51]:
# cost of predicted full-funnel campaign
branding_ad_cost = cpm/1000 * df_predicted %>% filter(campaign_type == 'full_funnel_predicted', ad_type == 'branding') %>% nrow
performance_ad_cost = cpa * df_predicted %>% filter(campaign_type == 'full_funnel_predicted', ad_type == 'performance') %>% pull(purchase) %>% sum
cost_full_funnel_predicted = branding_ad_cost + performance_ad_cost

print(paste("branding ad cost:", branding_ad_cost))
print(paste("performance ad cost:", performance_ad_cost))
print(paste("total cost:", cost_full_funnel_predicted))

[1] "branding ad cost: 417.12"
[1] "performance ad cost: 7770"
[1] "total cost: 8187.12"


In [52]:
# ROI of predicted full-funnel campaign
roi_full_funnel_predicted = ((df_predicted %>% filter(campaign_type == 'full_funnel_predicted') %>% pull(sales) %>% sum - 
                    df %>% filter(campaign_type == 'control') %>% pull(sales) %>% sum) - cost_full_funnel_predicted) / cost_full_funnel_predicted
print(paste("predicted full-funnel campaign ROI:", roi_full_funnel_predicted))

[1] "predicted full-funnel campaign ROI: 3.99565170658302"


##### low prediction accuracy (30%)

In [53]:
df_predicted = read.csv('data_predicted_low.csv')

In [54]:
df_predicted$stage_progress = ifelse(df_predicted$current_funnel_stage == df_predicted$next_funnel_stage, 0, 1)

In [55]:
df_predicted = rbind(df_predicted, 
                     df %>% mutate(current_funnel_stage_predicted = current_funnel_stage,
                                  next_funnel_stage_predicted = next_funnel_stage))
df_predicted = arrange(df_predicted, campaign_type, user_id, visit)

In [56]:
# control group as the baseline
df_predicted$campaign_type = as.factor(df_predicted$campaign_type)
df_predicted$campaign_type <- relevel(df_predicted$campaign_type, ref = 'control')

In [57]:
# no ad as the baseline
df_predicted$ad_type = as.factor(df_predicted$ad_type)
df_predicted$ad_type <- relevel(df_predicted$ad_type, ref = 'none')

In [58]:
# purchase probability as the outcome (all campaigns)
# intercept shows average probability without any ad
# data is aggregated from user-visit level to user level
model = lm(purchase ~ campaign_type, 
           df_predicted %>% 
           group_by(user_id) %>% 
           mutate(purchase = sum(purchase)) %>% 
           distinct(user_id, .keep_all = T))
summary(model)


Call:
lm(formula = purchase ~ campaign_type, data = df_predicted %>% 
    group_by(user_id) %>% mutate(purchase = sum(purchase)) %>% 
    distinct(user_id, .keep_all = T))

Residuals:
    Min      1Q  Median      3Q     Max 
-0.0953 -0.0834 -0.0638 -0.0430  0.9577 

Coefficients:
                                     Estimate Std. Error t value Pr(>|t|)    
(Intercept)                          0.043000   0.002506  17.162  < 2e-16 ***
campaign_typebrand_plus_performance  0.020800   0.003544   5.870 4.38e-09 ***
campaign_typebranding               -0.000700   0.003544  -0.198    0.843    
campaign_typefull_funnel             0.052300   0.003544  14.760  < 2e-16 ***
campaign_typefull_funnel_predicted   0.035800   0.003544  10.103  < 2e-16 ***
campaign_typeperformance             0.040400   0.003544  11.401  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.2506 on 59994 degrees of freedom
Multiple R-squared:  0.006337,	Adjusted R-sq

In [59]:
# purchase probability as the outcome (directly comparing two campaigns)
model = lm(purchase ~ campaign_type, 
           df_predicted %>% 
           group_by(user_id) %>% 
           mutate(purchase = sum(purchase)) %>% 
           distinct(user_id, .keep_all = T) %>%
           filter(campaign_type %in% c('full_funnel_predicted', 'full_funnel')))
summary(model)


Call:
lm(formula = purchase ~ campaign_type, data = df_predicted %>% 
    group_by(user_id) %>% mutate(purchase = sum(purchase)) %>% 
    distinct(user_id, .keep_all = T) %>% filter(campaign_type %in% 
    c("full_funnel_predicted", "full_funnel")))

Residuals:
    Min      1Q  Median      3Q     Max 
-0.0953 -0.0953 -0.0788 -0.0788  0.9212 

Coefficients:
                                    Estimate Std. Error t value Pr(>|t|)    
(Intercept)                         0.095300   0.002818   33.82  < 2e-16 ***
campaign_typefull_funnel_predicted -0.016500   0.003985   -4.14 3.48e-05 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.2818 on 19998 degrees of freedom
Multiple R-squared:  0.0008564,	Adjusted R-squared:  0.0008065 
F-statistic: 17.14 on 1 and 19998 DF,  p-value: 3.484e-05


In [60]:
# sales as the outcome (all campaigns)
# intercept shows average sales without any ad
# data is aggregated from user-visit level to user level
model = lm(sales ~ campaign_type, 
           df_predicted %>% 
           group_by(user_id) %>% 
           mutate(sales = sum(sales)) %>% 
           distinct(user_id, .keep_all = T))
summary(model)


Call:
lm(formula = sales ~ campaign_type, data = df_predicted %>% group_by(user_id) %>% 
    mutate(sales = sum(sales)) %>% distinct(user_id, .keep_all = T))

Residuals:
   Min     1Q Median     3Q    Max 
 -9.53  -8.34  -6.38  -4.30  95.77 

Coefficients:
                                    Estimate Std. Error t value Pr(>|t|)    
(Intercept)                           4.3000     0.2506  17.162  < 2e-16 ***
campaign_typebrand_plus_performance   2.0800     0.3543   5.870 4.38e-09 ***
campaign_typebranding                -0.0700     0.3543  -0.198    0.843    
campaign_typefull_funnel              5.2300     0.3543  14.760  < 2e-16 ***
campaign_typefull_funnel_predicted    3.5800     0.3543  10.103  < 2e-16 ***
campaign_typeperformance              4.0400     0.3543  11.401  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 25.06 on 59994 degrees of freedom
Multiple R-squared:  0.006337,	Adjusted R-squared:  0.006254 
F-statistic: 7

In [61]:
# sales as the outcome (directly comparing two campaigns)
model = lm(sales ~ campaign_type, 
           df_predicted %>% 
           group_by(user_id) %>% 
           mutate(sales = sum(sales)) %>% 
           distinct(user_id, .keep_all = T) %>%
           filter(campaign_type %in% c('full_funnel_predicted', 'full_funnel')))
summary(model)


Call:
lm(formula = sales ~ campaign_type, data = df_predicted %>% group_by(user_id) %>% 
    mutate(sales = sum(sales)) %>% distinct(user_id, .keep_all = T) %>% 
    filter(campaign_type %in% c("full_funnel_predicted", "full_funnel")))

Residuals:
   Min     1Q Median     3Q    Max 
 -9.53  -9.53  -7.88  -7.88  92.12 

Coefficients:
                                   Estimate Std. Error t value Pr(>|t|)    
(Intercept)                          9.5300     0.2818   33.82  < 2e-16 ***
campaign_typefull_funnel_predicted  -1.6500     0.3985   -4.14 3.48e-05 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 28.18 on 19998 degrees of freedom
Multiple R-squared:  0.0008564,	Adjusted R-squared:  0.0008065 
F-statistic: 17.14 on 1 and 19998 DF,  p-value: 3.484e-05


In [62]:
# cost of predicted full-funnel campaign
branding_ad_cost = cpm/1000 * df_predicted %>% filter(campaign_type == 'full_funnel_predicted', ad_type == 'branding') %>% nrow
performance_ad_cost = cpa * df_predicted %>% filter(campaign_type == 'full_funnel_predicted', ad_type == 'performance') %>% pull(purchase) %>% sum
cost_full_funnel_predicted = branding_ad_cost + performance_ad_cost

print(paste("branding ad cost:", branding_ad_cost))
print(paste("performance ad cost:", performance_ad_cost))
print(paste("total cost:", cost_full_funnel_predicted))

[1] "branding ad cost: 314.28"
[1] "performance ad cost: 6920"
[1] "total cost: 7234.28"


In [63]:
# ROI of predicted full-funnel campaign
roi_full_funnel_predicted = ((df_predicted %>% filter(campaign_type == 'full_funnel_predicted') %>% pull(sales) %>% sum - 
                    df %>% filter(campaign_type == 'control') %>% pull(sales) %>% sum) - cost_full_funnel_predicted) / cost_full_funnel_predicted
print(paste("predicted full-funnel campaign ROI:", roi_full_funnel_predicted))

[1] "predicted full-funnel campaign ROI: 3.9486610968887"
